In [1]:
import pandas as pd
import tmdbsimple as tmdb
import datetime
import bs4

with open('tmdb-api-key.txt', 'r') as f:
    tmdb.API_KEY = f.read().split('\n')[0]
    
db_type='movie'

In [2]:
search = tmdb.Search()
response = search.movie(query='Avengers: Endgame')
response['results'][0]

{'adult': False,
 'backdrop_path': '/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg',
 'genre_ids': [12, 878, 28],
 'id': 299534,
 'original_language': 'en',
 'original_title': 'Avengers: Endgame',
 'overview': "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.",
 'popularity': 292.968,
 'poster_path': '/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg',
 'release_date': '2019-04-24',
 'title': 'Avengers: Endgame',
 'video': False,
 'vote_average': 8.3,
 'vote_count': 17043}

In [15]:
search.tv(query='Friends')['results'][0]

{'backdrop_path': '/l0qVZIpXtIo7km9u5Yqh0nKPOr5.jpg',
 'first_air_date': '1994-09-22',
 'genre_ids': [35, 18],
 'id': 1668,
 'name': 'Friends',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'Friends',
 'overview': 'The misadventures of a group of friends as they navigate the pitfalls of work, life and love in Manhattan.',
 'popularity': 217.572,
 'poster_path': '/f496cm9enuEsZkSPzCwnTESEK5s.jpg',
 'vote_average': 8.4,
 'vote_count': 4022}

In [4]:
df = pd.read_csv('db_{}.csv'.format(db_type), encoding='utf-8')
df

,Title,Year,Rating,Poster,Quote,Type,Verified
0,The Prestige,2006,8.2,https://image.tmdb.org/t/p/w342/pvSESD7ujxWZwt...,NaN,live-action,y
1,Interstellar,2014,8.3,https://image.tmdb.org/t/p/w342/gEU2QniE6E77NI...,NaN,live-action,y
2,Arrival,2016,7.5,https://image.tmdb.org/t/p/w342/x2FJsf1ElAgr63...,NaN,live-action,y
3,Inception,2010,8.3,https://image.tmdb.org/t/p/w342/9gk7adHYeDvHkC...,NaN,live-action,y
4,Avengers: Endgame,2019,8.3,https://image.tmdb.org/t/p/w342/or06FN3Dka5tuk...,NaN,live-action,y
5,The Dark Knight,2008,8.5,https://image.tmdb.org/t/p/w342/qJ2tW6WMUDux91...,NaN,live-action,y
6,Your Name.,2016,8.6,https://image.tmdb.org/t/p/w342/q719jXXEzOoYap...,NaN,animation,y
7,Inglourious Basterds,2009,8.2,https://image.tmdb.org/t/p/w342/7sfbEnaARXDDhK...,NaN,live-action,y
8,Paprika,2006,7.8,https://image.tmdb.org/t/p/w342/dJI2FSs3qfEPW8...,NaN,animation,y
9,Inside Out,2015,7.9,https://image.tmdb.org/t/p/w342/lRHE0vzf3oYJrh...,NaN,animation,y


In [5]:
def update_db(df, db_type='movie'):
    for idx, movie_title in enumerate(df['Title']):
#         print(movie_title)
        if df.loc[idx, 'Verified'] == 'y':
            # skip already verified data
            continue
        if db_type == 'movie':
            response = search.movie(query=movie_title)['results'][0]
        elif db_type == 'tv':
            response = search.tv(query=movie_title)['results'][0]

        try:
            if db_type == 'movie':
                df.loc[idx, 'Title'] = response['title']
                df.loc[idx, 'Year'] = int(datetime.datetime.strptime(response['release_date'], '%Y-%m-%d').year)
            elif db_type == 'tv':
                df.loc[idx, 'Title'] = response['name']
                df.loc[idx, 'Year'] = int(datetime.datetime.strptime(response['first_air_date'], '%Y-%m-%d').year)
            df.loc[idx, 'Poster'] = 'https://image.tmdb.org/t/p/w342' + response['poster_path']
            df.loc[idx, 'Rating'] = response['vote_average']
#             print(movie_title, df.loc[idx, 'Year'])
        except Exception as e:
            print(movie_title)
            print("\t", e)
#         print(df)
    df['Year'] = df['Year'].astype(int)
    return df
        

df = update_db(df, db_type=db_type)
df

,Title,Year,Rating,Poster,Quote,Type,Verified
0,The Prestige,2006,8.2,https://image.tmdb.org/t/p/w342/pvSESD7ujxWZwt...,NaN,live-action,y
1,Interstellar,2014,8.3,https://image.tmdb.org/t/p/w342/gEU2QniE6E77NI...,NaN,live-action,y
2,Arrival,2016,7.5,https://image.tmdb.org/t/p/w342/x2FJsf1ElAgr63...,NaN,live-action,y
3,Inception,2010,8.3,https://image.tmdb.org/t/p/w342/9gk7adHYeDvHkC...,NaN,live-action,y
4,Avengers: Endgame,2019,8.3,https://image.tmdb.org/t/p/w342/or06FN3Dka5tuk...,NaN,live-action,y
5,The Dark Knight,2008,8.5,https://image.tmdb.org/t/p/w342/qJ2tW6WMUDux91...,NaN,live-action,y
6,Your Name.,2016,8.6,https://image.tmdb.org/t/p/w342/q719jXXEzOoYap...,NaN,animation,y
7,Inglourious Basterds,2009,8.2,https://image.tmdb.org/t/p/w342/7sfbEnaARXDDhK...,NaN,live-action,y
8,Paprika,2006,7.8,https://image.tmdb.org/t/p/w342/dJI2FSs3qfEPW8...,NaN,animation,y
9,Inside Out,2015,7.9,https://image.tmdb.org/t/p/w342/lRHE0vzf3oYJrh...,NaN,animation,y


In [6]:
response = search.tv(query='Battlestar Galactica')['results']
response

[{'backdrop_path': '/vLnYnApolGhlLsH7zmIOcI70tHu.jpg',
  'first_air_date': '2004-10-18',
  'genre_ids': [18, 10765],
  'id': 1972,
  'name': 'Battlestar Galactica',
  'origin_country': ['US', 'CA', 'GB'],
  'original_language': 'en',
  'original_name': 'Battlestar Galactica',
  'overview': 'When an old enemy, the Cylons, resurface and obliterate the 12 colonies, the crew of the aged Galactica protect a small civilian fleet - the last of humanity - as they journey toward the fabled 13th colony, Earth.',
  'popularity': 65.829,
  'poster_path': '/99PJSbcO2LeM10uOGWeFihNp77j.jpg',
  'vote_average': 8.3,
  'vote_count': 863},
 {'backdrop_path': '/dCuSp114IesW7NRrpJFqggitmdU.jpg',
  'first_air_date': '1978-09-17',
  'genre_ids': [18, 878],
  'id': 501,
  'name': 'Battlestar Galactica',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'Battlestar Galactica',
  'overview': 'When the 12 Colonies of Man are wiped out by a cybernetic race called the Cylons, Commander A

In [7]:
def data2tag(column_tag="nature", img="something.jpg", title="Mountain", info="Lorem ipsum dolor..", year='1990'):
    attributes = {'class': 'column ' + column_tag}
    new_div_tag = soup.new_tag("div", **attributes)
    attributes = {'class': 'content'}
    new_div_tag.insert(0, soup.new_tag("div", **attributes))
    new_div_tag.div.insert(0, soup.new_tag("img", alt="content", src=img, style="width:100%"))
    new_div_tag.div.insert(1, soup.new_tag("h4"))
    new_div_tag.div.h4.string='{} ({})'.format(title, year)
    new_div_tag.div.insert(2, soup.new_tag("p"))
    new_div_tag.div.p.string=info
    return new_div_tag

In [8]:
df.sort_values('Rating', ascending=False)

,Title,Year,Rating,Poster,Quote,Type,Verified
6,Your Name.,2016,8.6,https://image.tmdb.org/t/p/w342/q719jXXEzOoYap...,NaN,animation,y
5,The Dark Knight,2008,8.5,https://image.tmdb.org/t/p/w342/qJ2tW6WMUDux91...,NaN,live-action,y
3,Inception,2010,8.3,https://image.tmdb.org/t/p/w342/9gk7adHYeDvHkC...,NaN,live-action,y
4,Avengers: Endgame,2019,8.3,https://image.tmdb.org/t/p/w342/or06FN3Dka5tuk...,NaN,live-action,y
1,Interstellar,2014,8.3,https://image.tmdb.org/t/p/w342/gEU2QniE6E77NI...,NaN,live-action,y
0,The Prestige,2006,8.2,https://image.tmdb.org/t/p/w342/pvSESD7ujxWZwt...,NaN,live-action,y
7,Inglourious Basterds,2009,8.2,https://image.tmdb.org/t/p/w342/7sfbEnaARXDDhK...,NaN,live-action,y
10,WALL·E,2008,8.0,https://image.tmdb.org/t/p/w342/hbhFnRzzg6ZDmm...,NaN,animation,y
14,Like Stars on Earth,2007,8.0,https://image.tmdb.org/t/p/w342/wEHkOBcvEiNZpF...,NaN,live-action,y
15,3 Idiots,2009,7.9,https://image.tmdb.org/t/p/w342/k1iWMypGjk1b59...,NaN,live-action,y


In [9]:
with open("templates/{}.html".format(db_type)) as f:
    txt = f.read()
    soup = bs4.BeautifulSoup(txt)
template = soup.find("div", class_="row")
# template.append(new_div_tag)

for i, item in enumerate(df.iloc):
    tag = data2tag(column_tag=item["Type"], img=item["Poster"], title=item['Title'], info="", year=item["Year"])
    if i ==0: 
        template.div.replaceWith(tag)
    else:
        template.append(tag)

with open('index_{}.html'.format(db_type), 'w', encoding='utf-8') as f:
    f.write(str(soup.prettify()))
    
df.to_csv('db_{}.csv'.format(db_type), index=False)